# HOW TO CREATE A PROJECT from scratch

+ this notebook is a template to generate Backend-required files to view a project with the DataDiVR (preview or VR)
+ STEP 1 and the "create a graph" section contains a template graph writing a required format (json) to then use the generate-project functions of the DataDiVR backend

+ STEP 2 to actually generate BACKEND project files.

In [1]:
import networkx as nx
import json 
import os
import pandas as pd

# these are the two functions one needs to create a JSON file to upload and create the project in the backend 
import nx2json as nx2j 
import uploaderGraph as uG

C:\Users\chris\AppData\Local\Temp\ipykernel_22300\3520272987.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
G = nx.read_edgelist("teapot40_links.csv", delimiter=',', nodetype=int)
print("Number of nodes: ", len(G.nodes()))
print("Number of Links: ", len(G.edges()))

# ===============================================
# GRAPH NAME AND DESCRIPTION - a string each
# ===============================================

G.graph['projectname'] = "Teapot_X02"
G.graph['info'] = "A toy graph for testing purposes. Number of nodes: "+str(len(G.nodes()))+", Links: "+ str(len(G.edges()))+"."

Number of nodes:  51361
Number of Links:  102560


### create node
 anntotations

In [3]:
import random 
def generate_random_words(num):
    words = ["alpha", "beta", "gamma", "delta", "epsilon", "zeta", "theta", "lambda", "mu", "nu"]
    return random.sample(words, num)

# Create a list to hold annotations in JSON format
l_annotations_json = []

# Process each node in the graph
for g in G.nodes():
    # Generate random annotations
    annotations = {
        "annot1": generate_random_words(random.randint(3, 4)),
        "annot2": generate_random_words(random.randint(1, 2)),
        "annot3": generate_random_words(random.randint(2, 3))
    }
    
    l_annotations_json.append(annotations)

# Create a dictionary mapping nodes to their annotations
d_annotations = dict(zip(G.nodes(), l_annotations_json))

# Set the node attributes in the graph
nx.set_node_attributes(G, d_annotations, name="annotation")


In [4]:
nodepos = pd.read_csv("teapot40_nodes.csv", delimiter=',',header =None)
pos = dict(zip(G.nodes(), zip(nodepos[0], nodepos[1], nodepos[2])))

In [5]:
nx.set_node_attributes(G, pos, name="pos")

In [6]:
d_nodecolors_rgba = dict(zip(G.nodes(),[(255,0,0,140)]*len(G.nodes())))
nx.set_node_attributes(G, d_nodecolors_rgba, name="nodecolor")
l_linkcolors_rgba = (0,255,0,100)
d_linkcolors_rgba = dict(zip(G.edges(), [l_linkcolors_rgba]*len(G.edges())))
nx.set_edge_attributes(G, d_linkcolors_rgba, name="linkcolor")

In [7]:
# second layout where only a few links appear

G_1 = G.copy()

edges_to_remove = list(G.edges())[::2]
G_1.remove_edges_from(edges_to_remove)

print("Number of nodes: ", len(G_1.nodes()))
print("Number of Links: ", len(G_1.edges()))

d_linkcolors_rgba = dict(zip(G_1.edges(), [(255,255,0,200)]*len(G_1.edges())))
nx.set_edge_attributes(G_1, d_linkcolors_rgba, name="linkcolor")

Number of nodes:  51361
Number of Links:  51280


# CREATE A PROJECT WITH Nx.Graph object(s)

In [8]:
nx2j.create_project([G_1])

Creating Project...
Successfully created the directory static/projects/Teapot_X02 
Project created successfully.
